In [8]:
import pandas as pd
import numpy as np



In [ ]:
# Reading in Data

df_medal_tally = pd.read_csv('./data/Olympic_Medal_Tally_History.csv')
df_athlete_event_details = pd.read_csv('./data/Olympic_Athlete_Event_details.csv')


In [26]:
print(df_athlete_event_details.shape)

df_filtered = df_athlete_event_details[df_athlete_event_details["medal"].notna()]

print(df_filtered[df_filtered["medal"] == "Bronze"].shape)
print(df_filtered[df_filtered["medal"] == "Silver"].shape)
print(df_filtered[df_filtered["medal"] == "Gold"].shape)

display(df_filtered[df_filtered["pos"] == "3"][df_filtered["medal"] != "Bronze"])



(316834, 11)
(14939, 11)
(14676, 11)
(15072, 11)


C:\Users\wojta\AppData\Local\Temp\ipykernel_15508\3221180895.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  display(df_filtered[df_filtered["pos"] == "3"][df_filtered["medal"] != "Bronze"])


,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport
284348,1912 Summer Olympics,6,SWE,Athletics,"Decathlon, Men",56970,Charles Lomberg,76283,3,Silver,False
296935,1912 Summer Olympics,6,USA,Athletics,"Pentathlon, Men",57014,Jim Donahue,78331,3,Silver,False
